# Reading and Working with Time Series Data

In [1]:
import pandas as pd
import numpy as np

Pandas provides extensive functionality for writing data in a DataFrame. In this case we use `pd.read_fwf` to read a fixed-width-file:

In [2]:
data = pd.read_fwf("http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii")

In [3]:
data.head()

,1950,1,-0.60310E-01
0,1950,2,0.626810
1,1950,3,-0.008128
2,1950,4,0.555100
3,1950,5,0.071577
4,1950,6,0.538570


However, that does not look good. We clearly have time series data, more specifically it is monthly data. But we can specify that we want to parse the dates from the first two columns togehter with `parse_dates = [[0, 1]]`

In [4]:
data = pd.read_fwf("http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii", parse_dates = [[0, 1]], header=None)

In [5]:
data.head()

,0_1,2
0,1950-01-01,-0.060310
1,1950-02-01,0.626810
2,1950-03-01,-0.008128
3,1950-04-01,0.555100
4,1950-05-01,0.071577


Now I rename the columns and reset the index to the time column:

In [6]:
data.columns = ['Month', 'Value']
data.index = data.Month
data = data.drop('Month', 1)

In [7]:
data.head()

,Value
Month,
1950-01-01,-0.060310
1950-02-01,0.626810
1950-03-01,-0.008128
1950-04-01,0.555100
1950-05-01,0.071577


By parsing the dates pandas has automatically created a DatetimeIndex

In [8]:
data.index

DatetimeIndex(['1950-01-01', '1950-02-01', '1950-03-01', '1950-04-01',
               '1950-05-01', '1950-06-01', '1950-07-01', '1950-08-01',
               '1950-09-01', '1950-10-01',
               ...
               '2018-11-01', '2018-12-01', '2019-01-01', '2019-02-01',
               '2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01',
               '2019-07-01', '2019-08-01'],
              dtype='datetime64[ns]', name='Month', length=836, freq=None)

In [9]:
min(data.index)

Timestamp('1950-01-01 00:00:00')

In [10]:
max(data.index)

Timestamp('2019-08-01 00:00:00')

What's problematic about the index is that we are looking at a whole month as a time period, but the index specifies points in time (timestamps).

In [11]:
data['1951-11-11':'1951-11-12']

,Value
Month,


We can change the index to a PeriodIndex:

In [12]:
data_pd = data.to_period()

In [13]:
data_pd.index

PeriodIndex(['1950-01', '1950-02', '1950-03', '1950-04', '1950-05', '1950-06',
             '1950-07', '1950-08', '1950-09', '1950-10',
             ...
             '2018-11', '2018-12', '2019-01', '2019-02', '2019-03', '2019-04',
             '2019-05', '2019-06', '2019-07', '2019-08'],
            dtype='period[M]', name='Month', length=836, freq='M')

In [14]:
data_pd['1951-11-11':'1951-11-12']

,Value
Month,
1951-11,-0.068519


In [15]:
data_pd['1951-11-11':'1952-01-12']

,Value
Month,
1951-11,-0.068519
1951-12,1.987200
1952-01,0.368250


## Efficiency of loading time series data

How do various data loads perform?

In [16]:
import timeit

print("infer_datetime_format = True, no date parser")
%timeit pd.read_fwf("http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii", parse_dates = [[0, 1]], infer_datetime_format = True, header = None,)

print("infer_datetime_format = False, no date parser")
%timeit pd.read_fwf("http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii", parse_dates = [[0, 1]], infer_datetime_format = False, header = None,)

print("infer_datetime_format = True, date parser provided")
dateparse = lambda x, y: pd.datetime.strptime('%s-%s'%(x,y), '%Y-%m')
%timeit pd.read_fwf("http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii", parse_dates = [[0, 1]], infer_datetime_format = True, date_parser = dateparse,  header = None,)

print("infer_datetime_format = False, date parser provided")
dateparse = lambda x, y: pd.datetime.strptime('%s-%s'%(x,y), '%Y-%m')
%timeit pd.read_fwf("http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii", parse_dates = [[0, 1]], infer_datetime_format = False, date_parser = dateparse,  header = None,)

infer_datetime_format = True, no date parser
869 ms ± 55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
infer_datetime_format = False, no date parser
861 ms ± 45.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
infer_datetime_format = True, date parser provided
1.02 s ± 340 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
infer_datetime_format = False, date parser provided
856 ms ± 26.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Extract datetimes from exising DataFrame

When a DataFrame in a good format already exists it is really easy to create a DatetimeIndex from it:

In [17]:
df = pd.DataFrame({'year': [2015, 2016],'month': [2, 3],'day': [4, 5],'hour': [2, 3]})
df

,year,month,day,hour
0,2015,2,4,2
1,2016,3,5,3


In [18]:
pd.to_datetime(df)

0   2015-02-04 02:00:00
1   2016-03-05 03:00:00
dtype: datetime64[ns]

In [19]:
pd.to_datetime(df[['year', 'month', 'day']])

0   2015-02-04
1   2016-03-05
dtype: datetime64[ns]